<a href="https://colab.research.google.com/github/derzhavin3016/fashion_mnist/blob/master/fashion_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>